## Parse the grammar and consider only chord-related information

In [1]:
from pathlib import Path
import os

grammar_path = Path('../grammar/')
test_grammar = Path('../grammar/abc/abc_music21.lark')

all_grammars = {}
for path, dirs, files in os.walk(grammar_path):
    for file in files:
        if file.endswith('.lark'):
            grammar_dict = {}
            with open(Path(path) / file, 'r') as g:
                start = False
                for line in g:
                    if 'shorthand' in line:
                        start = True
                        next(g)
                        continue
                    if start:
                        if line[:1] != '/' and line != '\n' and '?' not in line:
                            label, shorthands = line.split(':')
                            shorts = [x.replace('\n', '').strip(' ').strip('"') for x in shorthands.split('|')]
                            grammar_dict[label] = shorts

            all_grammars[file] = grammar_dict

all_grammars

{'abc_music21.lark': {'major': ['', 'M', 'maj'],
  'minor': ['m', 'min'],
  'augmented': ['+', 'aug'],
  'diminished': ['dim', 'o'],
  'dominant_seventh': ['7', 'dom7'],
  'major_seventh': ['maj7', 'M7'],
  'minor_major_seventh': ['mM7', 'm#7', 'minmaj7'],
  'minor_seventh': ['m7', 'min7'],
  'augmented_major_seventh': ['+M7', 'augmaj7'],
  'augmented_seventh': ['7+', '+7', 'aug7'],
  'half_diminished_seventh': ['ø7', 'm7b5'],
  'diminished_seventh': ['o7', 'dim7'],
  'seventh_flat_five': ['dom7dim5'],
  'major_sixth': ['6'],
  'minor_sixth': ['m6', 'min6'],
  'major_ninth': ['M9', 'Maj9'],
  'dominant_ninth': ['9', 'dom9'],
  'minor_major_ninth': ['mM9', 'minmaj9'],
  'minor_ninth': ['m9', 'min9'],
  'augmented_major_ninth': ['+M9', 'augmaj9'],
  'augmented_dominant_ninth': ['9#5', '+9', 'aug9'],
  'half_diminished_ninth': ['ø9'],
  'half_diminished_minor_ninth': ['øb9'],
  'diminished_ninth': ['o9', 'dim9'],
  'diminished_minor_ninth': ['ob9', 'dimb9'],
  'dominant_11th': ['11', 'dom

## Get chord grades

In [ ]:
import sys
import pandas as pd

sys.path.append(os.path.join(os.getcwd(), '..', 'lark-converters'))
sys.path.append(os.path.join(os.getcwd(), '..'))

from lark_to_harte import MISSING_INTERVALS, HARTE_SHORTHAND_MAP
from music21 import harmony

print(harmony.CHORD_TYPES['major'][0].replace('-', 'b'))

for grammar_name, grammar_shorts in all_grammars.items():
    short_grades = []
    print(grammar_name, '\n')
    for short_name, short_variants in grammar_shorts.items():
        short_name = short_name.replace('_', '-')
        # print(short_name, MISSING_INTERVALS.keys())
        if short_name in harmony.CHORD_TYPES.keys():
            short_grades.append([short_name, short_variants, harmony.CHORD_TYPES[short_name][0].replace('-', 'b')])
        if short_name in MISSING_INTERVALS.keys():
            short_grades.append([short_name, short_variants, MISSING_INTERVALS[short_name].replace('-', 'b')])

        df = pd.DataFrame(short_grades, columns=['chord_name', 'shorthands', 'grades_harte'])
        writer = pd.ExcelWriter(f'validation_{grammar_name.rstrip(".lark")}.xlsx', engine='xlsxwriter')
        df.to_excel(writer, sheet_name=grammar_name, index=False)
        writer.save()

    print(short_grades)